In [1]:
from tokenizers import Tokenizer

tokenizer = Tokenizer.from_pretrained("LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct")

/home/ksi/ai/spam/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from pandas import pandas as pd

data_train = pd.read_json("dataset-train-spm.json")
data_train["ids"] = data_train["message"].apply(lambda x: tokenizer.encode(x).ids)

X_train = data_train["ids"]
Y_train = data_train["type"]
tokenizer.get_vocab_size()

102400

In [3]:
data_train

,message,type,ids
0,강릉축협하나로마트 입암점 - - 강릉축협하나로마트 입암점밴드가입: https://b...,0,"[43656, 3236, 7059, 7027, 715, 5285, 1110, 488..."
1,지방시 뷰티 샘플링 및 메이크업 서비스 이벤트칙칙하고 생기 잃은 눈가 피부를 개선해...,0,"[20271, 954, 13547, 19006, 3890, 2425, 12425, ..."
2,<< ln.run/vb_ z >>적어 가입시교촌무슨이유든노재쟤뉴비 % all 죵목 ...,1,"[35642, 17146, 375, 24847, 376, 38555, 424, 17..."
3,언뉘들 많은 곳~이쁜이들이 기다리고 있어요연락주세요 !!llne:x x,1,"[2833, 78491, 1371, 1180, 732, 1763, 455, 3673..."
4,손흥민 응원만 하 면 3 만원 톳넘 5 처넌 까고 흥민이 골 넣으면 배달의민족 3만...,1,"[75981, 11204, 1043, 691, 928, 582, 380, 926, ..."
...,...,...,...
14532,오창정육점 / . 일요일 정상영업 합니다 만원 정육 구매시 선물 증정 합니다한돈 암...,0,"[1235, 3239, 1002, 2291, 1647, 841, 582, 375, ..."
14533,cure-tv 주말 신규 방문 편의점 쿠폰 5만추가지급 v카카오v id : xen44,1,"[428, 1025, 374, 14437, 5677, 10145, 3325, 141..."
14534,<< 마포 에피트 어바닉 >> 최고경쟁률 : 잔여세대 선착순 줍줍!!# 입지환경 호...,0,"[35642, 23286, 26201, 1067, 767, 1987, 8777, 1..."
14535,"[하나은행] 년 은행달력 증정 이벤트 배 현님! 년 은행달력, 전담직원이 다시 한번...",0,"[420, 7027, 9906, 422, 1393, 5610, 2230, 1398,..."


In [4]:
from tf_keras.preprocessing.sequence import pad_sequences

max_len = 200
X_train = pad_sequences(X_train, maxlen=max_len)

2025-06-05 12:18:01.684541: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-05 12:18:01.697793: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-05 12:18:01.726400: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-05 12:18:01.772067: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749093481.857615  715446 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749093481.88

In [5]:
from tf_keras.layers import Dense, Conv1D, GlobalMaxPooling1D, Embedding, Dropout, MaxPooling1D
from tf_keras.models import Sequential
from tf_keras.callbacks import EarlyStopping, ModelCheckpoint

embedding_dim = 32
num_filters = 128
kernel_size = 20
dropout_ratio = 0.3

model = Sequential()
model.add(Embedding(tokenizer.get_vocab_size(), embedding_dim))
model.add(Dropout(dropout_ratio))
model.add(Conv1D(num_filters, kernel_size, padding='valid', activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dropout(dropout_ratio))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
mc = ModelCheckpoint('cnn-spm.keras', monitor = 'val_acc', mode='max', verbose=1, save_best_only=True)

history = model.fit(X_train, Y_train, epochs=20, batch_size=64, validation_split=0.2, callbacks=[es, mc])


2025-06-05 12:18:19.818472: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Epoch 1/20
182/182 [==============================] - ETA: 0s - loss: 0.3085 - acc: 0.8546
Epoch 1: val_acc improved from -inf to 0.97937, saving model to cnn-spm.keras
182/182 [==============================] - 39s 186ms/step - loss: 0.3085 - acc: 0.8546 - val_loss: 0.0626 - val_acc: 0.9794
Epoch 2/20
182/182 [==============================] - ETA: 0s - loss: 0.0382 - acc: 0.9882
Epoch 2: val_acc improved from 0.97937 to 0.98762, saving model to cnn-spm.keras
182/182 [==============================] - 32s 175ms/step - loss: 0.0382 - acc: 0.9882 - val_loss: 0.0388 - val_acc: 0.9876
Epoch 3/20
182/182 [==============================] - ETA: 0s - loss: 0.0149 - acc: 0.9959
Epoch 3: val_acc improved from 0.98762 to 0.98900, saving model to cnn-spm.keras
182/182 [==============================] - 31s 168ms/step - loss: 0.0149 - acc: 0.9959 - val_loss: 0.0360 - val_acc: 0.9890
Epoch 4/20
182/182 [==============================] - ETA: 0s - loss: 0.0072 - acc: 0.9984
Epoch 4: val_acc did not